In [114]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import os, fnmatch
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures 
import seaborn as sns

import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from PIL import Image
%matplotlib inline

In [115]:
df = pd.read_csv("astro_objects.csv")

In [116]:
df.head()

,Unnamed: 0,plate,object_id,ra,dec,mag_B,mag_R,spectrum_class,spectrum_length,red_bord_position,9,10,11,12,13,14,15,16,fits_path
0,0,fbs0005,DFBSJ142114.64+425823.6,14:21:14.640,+42:58:23.63,13.86,13.15,OK,101,16,-2.189981e-11,-3.665937e-14,-9.373043e-12,-1.532294e-09,1.024176e-11,-2.280213e-14,2.109259e-17,-6.999758e-21,specimages/fbs0005-DFBSJ142114.64+425823.6.fits
1,1,fbs0005,DFBSJ142122.01+425921.3,14:21:22.012,+42:59:21.37,12.85,12.20,OK,106,13,-3.792762e-11,-3.647929e-14,-1.208479e-11,-1.865856e-09,1.801383e-11,-4.706942e-14,4.814870e-17,-1.724178e-20,specimages/fbs0005-DFBSJ142122.01+425921.3.fits
2,2,fbs0005,DFBSJ142125.68+425138.2,14:21:25.688,+42:51:38.29,16.38,14.85,NL,83,17,-1.815388e-11,-1.309881e-14,-3.936086e-12,-6.570811e-10,3.287456e-12,-6.007407e-15,4.796814e-18,-1.419620e-21,specimages/fbs0005-DFBSJ142125.68+425138.2.fits
3,3,fbs0005,DFBSJ142126.80+425301.4,14:21:26.802,+42:53:01.40,14.45,13.57,NL,120,17,-4.077909e-11,-2.942382e-14,-8.841611e-12,-1.475995e-09,8.681218e-12,-1.793400e-14,1.573814e-17,-5.009265e-21,specimages/fbs0005-DFBSJ142126.80+425301.4.fits
4,4,fbs0005,DFBSJ142141.43+430014.5,14:21:41.436,+43:00:14.59,16.39,16.47,NL,90,16,-1.024508e-12,-1.714980e-15,-4.384846e-13,-7.168274e-11,5.490072e-13,-1.330955e-15,1.317058e-18,-4.632790e-22,specimages/fbs0005-DFBSJ142141.43+430014.5.fits


In [117]:
fits_path = df["fits_path"].tolist()
jpeg_paths = []

In [118]:
for i in range(len(fits_path)):
    # height = data.shape[0]
    # width = data.shape[1]
    fitsfilename = fits_path[i]
    data = fits.open(fitsfilename)[0].data
    vmin = np.min(data)
    vmax = np.max(data)

    # print(data)

    # print(vmin)
    # print(vmax)

    data[data > vmax] = vmax
    data[data < vmin] = vmin
    # Scale data to range [0, 1] 
    data = (data - vmin)/(vmax - vmin)
    # Convert to 8-bit integer  
    data = (255*data).astype(np.uint8)
    # Invert y axis
    data = data[::-1, :]

    # Create image from data array and save as jpg
    image = Image.fromarray(data, 'L')
    imagename = fitsfilename.replace('.fits', '.jpg')
    imagename = imagename.replace("specimages","specimagesjpeg")
    jpeg_paths.append(imagename)
    image.save(imagename)

#     plt.figure()
#     plt.imshow(data, cmap='gray')
#     plt.colorbar()


In [119]:
jpeg_paths

['specimagesjpeg/fbs0005-DFBSJ142114.64+425823.6.jpg',
 'specimagesjpeg/fbs0005-DFBSJ142122.01+425921.3.jpg',
 'specimagesjpeg/fbs0005-DFBSJ142125.68+425138.2.jpg',
 'specimagesjpeg/fbs0005-DFBSJ142126.80+425301.4.jpg',
 'specimagesjpeg/fbs0005-DFBSJ142141.43+430014.5.jpg',
 'specimagesjpeg/fbs0005-DFBSJ142145.60+425741.8.jpg',
 'specimagesjpeg/fbs0006-DFBSJ144118.92+425749.2.jpg',
 'specimagesjpeg/fbs0006-DFBSJ144119.77+430129.7.jpg',
 'specimagesjpeg/fbs0006-DFBSJ144124.90+425948.6.jpg',
 'specimagesjpeg/fbs0006-DFBSJ144129.51+425736.3.jpg',
 'specimagesjpeg/fbs0006-DFBSJ144135.00+430003.3.jpg',
 'specimagesjpeg/fbs0006-DFBSJ144136.84+425315.9.jpg',
 'specimagesjpeg/fbs0006-DFBSJ144144.19+425739.1.jpg',
 'specimagesjpeg/fbs0006-DFBSJ144149.19+425811.0.jpg',
 'specimagesjpeg/fbs0006-DFBSJ144150.64+430212.2.jpg',
 'specimagesjpeg/fbs0006-DFBSJ144154.62+425758.8.jpg',
 'specimagesjpeg/fbs0007-DFBSJ150110.13+425902.6.jpg',
 'specimagesjpeg/fbs0007-DFBSJ150111.23+425941.7.jpg',
 'specimag

In [120]:
df["jpeg_path"] = pd.Series(jpeg_paths)

In [121]:
df.head(10)

,Unnamed: 0,plate,object_id,ra,dec,mag_B,mag_R,spectrum_class,spectrum_length,red_bord_position,9,10,11,12,13,14,15,16,fits_path,jpeg_path
0,0,fbs0005,DFBSJ142114.64+425823.6,14:21:14.640,+42:58:23.63,13.86,13.15,OK,101,16,-2.189981e-11,-3.665937e-14,-9.373043e-12,-1.532294e-09,1.024176e-11,-2.280213e-14,2.109259e-17,-6.999758e-21,specimages/fbs0005-DFBSJ142114.64+425823.6.fits,specimagesjpeg/fbs0005-DFBSJ142114.64+425823.6...
1,1,fbs0005,DFBSJ142122.01+425921.3,14:21:22.012,+42:59:21.37,12.85,12.20,OK,106,13,-3.792762e-11,-3.647929e-14,-1.208479e-11,-1.865856e-09,1.801383e-11,-4.706942e-14,4.814870e-17,-1.724178e-20,specimages/fbs0005-DFBSJ142122.01+425921.3.fits,specimagesjpeg/fbs0005-DFBSJ142122.01+425921.3...
2,2,fbs0005,DFBSJ142125.68+425138.2,14:21:25.688,+42:51:38.29,16.38,14.85,NL,83,17,-1.815388e-11,-1.309881e-14,-3.936086e-12,-6.570811e-10,3.287456e-12,-6.007407e-15,4.796814e-18,-1.419620e-21,specimages/fbs0005-DFBSJ142125.68+425138.2.fits,specimagesjpeg/fbs0005-DFBSJ142125.68+425138.2...
3,3,fbs0005,DFBSJ142126.80+425301.4,14:21:26.802,+42:53:01.40,14.45,13.57,NL,120,17,-4.077909e-11,-2.942382e-14,-8.841611e-12,-1.475995e-09,8.681218e-12,-1.793400e-14,1.573814e-17,-5.009265e-21,specimages/fbs0005-DFBSJ142126.80+425301.4.fits,specimagesjpeg/fbs0005-DFBSJ142126.80+425301.4...
4,4,fbs0005,DFBSJ142141.43+430014.5,14:21:41.436,+43:00:14.59,16.39,16.47,NL,90,16,-1.024508e-12,-1.714980e-15,-4.384846e-13,-7.168274e-11,5.490072e-13,-1.330955e-15,1.317058e-18,-4.632790e-22,specimages/fbs0005-DFBSJ142141.43+430014.5.fits,specimagesjpeg/fbs0005-DFBSJ142141.43+430014.5...
5,5,fbs0005,DFBSJ142145.60+425741.8,14:21:45.600,+42:57:41.81,17.01,16.58,U,27,18,-1.000002e-11,-2.660709e-15,-1.032694e-12,-1.762539e-10,8.974909e-13,-1.686491e-15,1.382008e-18,-4.168996e-22,specimages/fbs0005-DFBSJ142145.60+425741.8.fits,specimagesjpeg/fbs0005-DFBSJ142145.60+425741.8...
6,6,fbs0006,DFBSJ144118.92+425749.2,14:41:18.927,+42:57:49.27,15.95,14.96,U,116,16,-1.320912e-11,-2.211156e-14,-5.653477e-12,-9.242263e-10,5.038040e-12,-1.000580e-14,8.614502e-18,-2.723474e-21,specimages/fbs0006-DFBSJ144118.92+425749.2.fits,specimagesjpeg/fbs0006-DFBSJ144118.92+425749.2...
7,7,fbs0006,DFBSJ144119.77+430129.7,14:41:19.777,+43:01:29.72,12.38,12.04,OK,120,10,7.789383e-10,8.888174e-13,1.982672e-10,2.911844e-08,-1.420003e-10,2.711751e-13,-2.377857e-16,8.004468e-20,specimages/fbs0006-DFBSJ144119.77+430129.7.fits,specimagesjpeg/fbs0006-DFBSJ144119.77+430129.7...
8,8,fbs0006,DFBSJ144124.90+425948.6,14:41:24.903,+42:59:48.65,16.77,15.25,U,102,17,-2.350397e-11,-1.695913e-14,-5.096079e-12,-8.507273e-10,4.366447e-12,-8.256987e-15,6.814367e-18,-2.073816e-21,specimages/fbs0006-DFBSJ144124.90+425948.6.fits,specimagesjpeg/fbs0006-DFBSJ144124.90+425948.6...
9,9,fbs0006,DFBSJ144129.51+425736.3,14:41:29.512,+42:57:36.34,16.14,15.48,U,81,18,-6.568866e-12,-1.747772e-15,-6.783579e-13,-1.157774e-10,8.252546e-13,-1.840639e-15,1.666661e-18,-5.352658e-22,specimages/fbs0006-DFBSJ144129.51+425736.3.fits,specimagesjpeg/fbs0006-DFBSJ144129.51+425736.3...


In [123]:
df.to_csv("astro_objects.csv")